In [1]:
!pip install pandas 
!pip install numpy 
!pip install spacy
!pip install -U scikit-learn
!pip install seaborn
!pip install transformers

/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)


In [7]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from utils.emfd import *
from utils.spacy_preprocessor import SpaCyPreProcessor
import pickle
from tqdm.notebook import tqdm
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [2]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
# text = "Good night 😊"
# text = preprocess(text)
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)
# scores = output[0][0].detach().numpy()
# scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

In [4]:
#save pre trained model

model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained("models/" + MODEL)

2023-07-05 19:33:12.331774: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-05 19:33:12.336564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-05 19:33:12.339351: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
#load model

model = TFAutoModelForSequenceClassification.from_pretrained("models/" + MODEL)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at models/cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
davidson = pd.read_csv("datasets/edited/davidson_cleaned.csv")

In [ ]:
def get_scores(scores):
    score = dict()
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        score[l] = np.round(float(s), 4)

    return score


davidson["positive"] = 0
davidson["neutral"] = 0
davidson["negative"] = 0


for i in tqdm(davidson.index):
    text = davidson["cleaned_tweets"][i]
    encoded_input = tokenizer(text, return_tensors='tf')
    output = model(**encoded_input)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    dict_scores = get_scores(scores)
    davidson["positive"][i] = dict_scores["positive"]
    davidson["neutral"][i] = dict_scores["neutral"]
    davidson["negative"][i] = dict_scores["negative"]

In [11]:
def get_scores(scores):
    score = dict()
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        score[l] = np.round(float(s), 4)

    return score


davidson["positive"] = 0
davidson["neutral"] = 0
davidson["negative"] = 0


for i in tqdm(davidson.index):
    text = davidson["tweet"][i]
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    dict_scores = get_scores(scores)
    davidson["positive"][i] = dict_scores["positive"]
    davidson["neutral"][i] = dict_scores["neutral"]
    davidson["negative"][i] = dict_scores["negative"]

davidson

  0%|          | 0/24783 [00:00<?, ?it/s]

/tmp/ipykernel_20148/2558099294.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  davidson["positive"][i] = dict_scores["positive"]
/tmp/ipykernel_20148/2558099294.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  davidson["neutral"][i] = dict_scores["neutral"]
/tmp/ipykernel_20148/2558099294.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  davidson["negative"][i] = dict_scores["negative"]


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,care.virtue,care.vice,fairness.virtue,...,loyalty.virtue,loyalty.vice,authority.virtue,authority.vice,sanctity.virtue,sanctity.vice,cleaned_tweets,positive,neutral,negative
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,0,0,0,...,0,0,0,0,1,1,rt mayasolovely woman complain clean house am...,0.2071,0.5617,0.2312
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,0,0,...,0,0,0,0,0,0,rt mleew boy dat cold tyga dwn bad cuffin d...,0.0076,0.0654,0.9270
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,0,0,...,0,0,0,0,0,2,rt urkindofbrand dawg rt sbaby life fuck b...,0.0382,0.1689,0.7929
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,0,0,...,0,0,0,0,0,0,rt c g anderson viva based look like tranny,0.0160,0.1128,0.8712
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,0,0,...,0,0,0,0,0,1,rt shenikaroberts shit hear true faker bitch ...,0.0304,0.1669,0.8027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0,0,0,...,0,0,0,0,1,1,muthaf in lie lifeasking pearls c...,0.0064,0.0448,0.9489
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an...",0,0,0,...,0,0,0,0,0,0,go break wrong heart baby drive redneck crazy,0.0107,0.1059,0.8834
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...,0,0,0,...,0,0,0,0,0,0,young buck wanna eat dat nigguh like be nt fuc...,0.1603,0.3212,0.5186
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies,0,0,0,...,0,0,0,0,0,0,youu got wild bitch tellin lie,0.0123,0.1872,0.8005


In [6]:
davidson

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,care.virtue,care.vice,fairness.virtue,...,loyalty.virtue,loyalty.vice,authority.virtue,authority.vice,sanctity.virtue,sanctity.vice,cleaned_tweets,positive,neutral,negative
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,0,0,0,...,0,0,0,0,1,1,rt mayasolovely woman complain clean house am...,0.2071,0.5617,0.2312
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,0,0,...,0,0,0,0,0,0,rt mleew boy dat cold tyga dwn bad cuffin d...,0.0076,0.0654,0.9270
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,0,0,...,0,0,0,0,0,2,rt urkindofbrand dawg rt sbaby life fuck b...,0.0382,0.1689,0.7929
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,0,0,...,0,0,0,0,0,0,rt c g anderson viva based look like tranny,0.0160,0.1128,0.8712
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,0,0,...,0,0,0,0,0,1,rt shenikaroberts shit hear true faker bitch ...,0.0304,0.1669,0.8027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0,0,0,...,0,0,0,0,1,1,muthaf in lie lifeasking pearls c...,0.0000,0.0000,0.0000
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an...",0,0,0,...,0,0,0,0,0,0,go break wrong heart baby drive redneck crazy,0.0000,0.0000,0.0000
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...,0,0,0,...,0,0,0,0,0,0,young buck wanna eat dat nigguh like be nt fuc...,0.0000,0.0000,0.0000
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies,0,0,0,...,0,0,0,0,0,0,youu got wild bitch tellin lie,0.0000,0.0000,0.0000


In [ ]:
davidson.to_csv("datasets/edited/davidson_sentiment_score.csv", index = False, header = True)